In [28]:
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import pandas as pd
import datetime as dt
import time
from datetime import datetime

In [29]:
# OANDA account details
access_token = "cf239e3c067ad4c1746757b6d7ed79e4-942b563fc891c05ca686299a353185e8"
accountID = "101-001-25542127-001"
#api = oandapyV20.API(access_token=access_token, environment="practice")
try:
    client = oandapyV20.API(access_token=access_token, environment="practice")
except V20Error as e:
    print("Error: {}".format(e))
    

In [30]:
# specify the instrument to trade
instrument = "EUR_CHF"

In [31]:
# set the order parameters
order_quantity = 1000 # start with 10k units
total_order_quantity = 100000 # the total order quantity
current_executed_quantity = 0 # keep track of how many units have been executed so far
non_executed_quantity = 0 # keep track of the units that were not executed in the previous window
average_execution_price = 0 # initialize the average execution price to 0
total_average_execution_price = 0 # initialize the total average execution price to 0

# setting the required time frames
first_execution_start_time = dt.datetime.now().replace(hour=15, minute=30, second=0, microsecond=0)
first_execution_end_time = dt.datetime.now().replace(hour=18, minute=30, second=0, microsecond=0)

second_execution_start_time = dt.datetime.now().replace(hour=19, minute=0, second=0, microsecond=0)
second_execution_end_time = dt.datetime.now().replace(hour=22, minute=0, second=0, microsecond=0)

third_execution_start_time = dt.datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
third_execution_end_time = dt.datetime.now().replace(hour=1, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) 

fourth_execution_start_time = dt.datetime.now().replace(hour=3, minute=0, second=0, microsecond=0) 
fourth_execution_end_time = dt.datetime.now().replace(hour=6, minute=0, second=0, microsecond=0) 

In [32]:
# define a function to execute orders
def execute_order(quantity, trades):
    # create a market order request
    mo = MarketOrderRequest(
        instrument=instrument,
        units=quantity,
        takeProfitOnFill=None,
        stopLossOnFill=None
    )
    # send the order request
    r = orders.OrderCreate(accountID, data=mo.data)
    response = client.request(r)
    if "orderFillTransaction" in response:
        trades.append({
            "Timestamp": dt.datetime.now(), 
            "Order_ID": float(response["orderFillTransaction"]["orderID"]),
            "Instrument": instrument,
            "Price": float(response["orderFillTransaction"]["price"]),
        })
    else:
        # Handle the case where the order is not filled
        if "orderCancelTransaction" in response:
            print("Order canceled due to:", response["orderCancelTransaction"]["reason"])
        else:
            print("No cancel transaction found.")

In [33]:
# make a function to get price that can be used to compare with the average price before execution takes place
def get_rate():
    price_request = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    response = client.request(price_request)
    return float(response['prices'][0]['asks'][0]['price'])

In [34]:
# execute the first window
while dt.datetime.now() < first_execution_start_time:
    time.sleep(1)

batch1_prices = []  # create a list to store the prices
while dt.datetime.now() >= first_execution_start_time and dt.datetime.now() <= first_execution_end_time:
    for i in range(20): # 20 iterations to execute 10k units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.2:       
            response = execute_order(order_quantity, batch1_prices)
            current_executed_quantity += order_quantity
            print("execute order quantity:",order_quantity,"current executed:",current_executed_quantity)
        else:
            break
            print("break")
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

execute order quantity: 1000 current executed: 1000
Successful execution for 0 th interval
execute order quantity: 1000 current executed: 2000
Successful execution for 1 th interval
execute order quantity: 1000 current executed: 3000
Successful execution for 2 th interval
execute order quantity: 1000 current executed: 4000
Successful execution for 3 th interval
execute order quantity: 1000 current executed: 5000
Successful execution for 4 th interval
execute order quantity: 1000 current executed: 6000
Successful execution for 5 th interval
execute order quantity: 1000 current executed: 7000
Successful execution for 6 th interval
execute order quantity: 1000 current executed: 8000
Successful execution for 7 th interval
execute order quantity: 1000 current executed: 9000
Successful execution for 8 th interval
execute order quantity: 1000 current executed: 10000
Successful execution for 9 th interval
execute order quantity: 1000 current executed: 11000
Successful execution for 10 th inter

KeyboardInterrupt: 

In [35]:
# convert the list of prices into a DataFrame
df1 = pd.DataFrame(batch1_prices)
df1

,Timestamp,Order_ID,Instrument,Price
0,2023-05-17 22:08:55.952409,103.0,EUR_CHF,0.97436
1,2023-05-17 22:09:01.011896,105.0,EUR_CHF,0.97436
2,2023-05-17 22:09:06.059558,107.0,EUR_CHF,0.97435
3,2023-05-17 22:09:11.116799,109.0,EUR_CHF,0.97437
4,2023-05-17 22:09:16.172866,111.0,EUR_CHF,0.97437
5,2023-05-17 22:09:21.233375,113.0,EUR_CHF,0.97436
6,2023-05-17 22:09:26.292793,115.0,EUR_CHF,0.97436
7,2023-05-17 22:09:31.345041,117.0,EUR_CHF,0.97436
8,2023-05-17 22:09:36.416001,119.0,EUR_CHF,0.97437
9,2023-05-17 22:09:41.472504,121.0,EUR_CHF,0.97434


In [25]:
average_execution_price1 = df1['Price'].sum() / len(df1)
print(average_execution_price1)
print(current_executed_quantity)
df1.to_csv("Long_Interval_1.csv", index=False)

0.974365
20000


In [36]:
# execute the second window
while dt.datetime.now() < second_execution_start_time:
    time.sleep(1)

batch2_prices = []  # create a list to store the prices

while dt.datetime.now() >= second_execution_start_time and dt.datetime.now() <= second_execution_end_time:
    for i in range(30): # 30 iterations to execute 1k units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.5:
            if get_rate() >= average_execution_price1:
                execute_order(order_quantity, batch2_prices)
                current_executed_quantity += order_quantity
                print("execute order quantity:",order_quantity,"current executed:",current_executed_quantity)
            else:
                print("Price is less than average price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

execute order quantity: 1000 current executed: 21000
Successful execution for 0 th interval
execute order quantity: 1000 current executed: 22000
Successful execution for 1 th interval
execute order quantity: 1000 current executed: 23000
Successful execution for 2 th interval
execute order quantity: 1000 current executed: 24000
Successful execution for 3 th interval
execute order quantity: 1000 current executed: 25000
Successful execution for 4 th interval
execute order quantity: 1000 current executed: 26000
Successful execution for 5 th interval
execute order quantity: 1000 current executed: 27000
Successful execution for 6 th interval
execute order quantity: 1000 current executed: 28000
Successful execution for 7 th interval
execute order quantity: 1000 current executed: 29000
Successful execution for 8 th interval
execute order quantity: 1000 current executed: 30000
Successful execution for 9 th interval
execute order quantity: 1000 current executed: 31000
Successful execution for 10

KeyboardInterrupt: 

In [37]:
# convert the list of prices into a DataFrame
df2 = pd.DataFrame(batch2_prices)
df2

,Timestamp,Order_ID,Instrument,Price
0,2023-05-17 22:11:06.384462,143.0,EUR_CHF,0.97437
1,2023-05-17 22:11:11.497433,145.0,EUR_CHF,0.97438
2,2023-05-17 22:11:16.608003,147.0,EUR_CHF,0.97439
3,2023-05-17 22:11:21.705292,149.0,EUR_CHF,0.97438
4,2023-05-17 22:11:26.802215,151.0,EUR_CHF,0.97439
5,2023-05-17 22:11:31.902020,153.0,EUR_CHF,0.97438
6,2023-05-17 22:11:37.004152,155.0,EUR_CHF,0.97438
7,2023-05-17 22:11:42.117221,157.0,EUR_CHF,0.97440
8,2023-05-17 22:11:47.238061,159.0,EUR_CHF,0.97441
9,2023-05-17 22:11:52.332257,161.0,EUR_CHF,0.97439


In [39]:
# concatenate the two dataframes vertically
new_df = pd.concat([df1, df2], axis=0)
# calculate the total average execution price
average_execution_price2 = new_df['Price'].sum() / len(new_df)

print(average_execution_price2)
print(current_executed_quantity)
non_executed_quantity2 = total_order_quantity * 0.5 - current_executed_quantity
print(non_executed_quantity2)
df2.to_csv("Long_Interval_2.csv", index=False)

0.9743900000000001
50000
0.0


In [40]:
# execute the third window
while dt.datetime.now() < third_execution_start_time:
    time.sleep(1)

batch3_prices = []  # create a list to store the prices

while dt.datetime.now() >= third_execution_start_time and dt.datetime.now() <= third_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity2 + (total_order_quantity * 0.2)) / 20)
    for i in range(20): # 20 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.7:
            if get_rate() >= average_execution_price2:
                response = execute_order(order_quantity, batch3_prices)
                current_executed_quantity += order_quantity
                print("execute order quantity:",order_quantity,"current executed:",current_executed_quantity)
            else:
                print("Price is less than average price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

execute order quantity: 1000 current executed: 51000
Successful execution for 0 th interval
execute order quantity: 1000 current executed: 52000
Successful execution for 1 th interval
execute order quantity: 1000 current executed: 53000
Successful execution for 2 th interval
execute order quantity: 1000 current executed: 54000
Successful execution for 3 th interval
execute order quantity: 1000 current executed: 55000
Successful execution for 4 th interval
execute order quantity: 1000 current executed: 56000
Successful execution for 5 th interval
execute order quantity: 1000 current executed: 57000
Successful execution for 6 th interval
execute order quantity: 1000 current executed: 58000
Successful execution for 7 th interval
execute order quantity: 1000 current executed: 59000
Successful execution for 8 th interval
Price is less than average price
Successful execution for 9 th interval
Price is less than average price
Successful execution for 10 th interval
execute order quantity: 100

KeyboardInterrupt: 

In [41]:
# convert the list of prices into a DataFrame
df3 = pd.DataFrame(batch3_prices)
df3

,Timestamp,Order_ID,Instrument,Price
0,2023-05-17 22:14:32.286243,203.0,EUR_CHF,0.97442
1,2023-05-17 22:14:37.390647,205.0,EUR_CHF,0.97443
2,2023-05-17 22:14:42.492729,207.0,EUR_CHF,0.97443
3,2023-05-17 22:14:47.586203,209.0,EUR_CHF,0.97443
4,2023-05-17 22:14:52.680367,211.0,EUR_CHF,0.97443
5,2023-05-17 22:14:57.794394,213.0,EUR_CHF,0.97445
6,2023-05-17 22:15:02.897760,215.0,EUR_CHF,0.97444
7,2023-05-17 22:15:08.002384,217.0,EUR_CHF,0.97444
8,2023-05-17 22:15:13.116841,219.0,EUR_CHF,0.97444
9,2023-05-17 22:15:28.333076,221.0,EUR_CHF,0.97440


In [42]:
# concatenate the two dataframes vertically
updated_df = pd.concat([new_df, df3], axis=0)
# calculate the total average execution price
average_execution_price3 = updated_df['Price'].sum() / len(updated_df)

print(average_execution_price3)
print(current_executed_quantity)
non_executed_quantity3 = total_order_quantity * 0.7 - current_executed_quantity
print(non_executed_quantity3)

df3.to_csv("Long_Interval_3.csv", index=False)

0.9744021428571431
70000
0.0


In [43]:
# execute the fourth window
while dt.datetime.now() < fourth_execution_start_time:
    time.sleep(1)

batch4_prices = []  # create a list to store the prices

while dt.datetime.now() >= fourth_execution_start_time and dt.datetime.now() <= fourth_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity3 + (total_order_quantity * 0.3)) / 30)
    for i in range(30): # 30 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity:
            if get_rate() >= average_execution_price3:
                response = execute_order(order_quantity, batch4_prices)
                current_executed_quantity += order_quantity
                print("execute order quantity:",order_quantity,"current executed:",current_executed_quantity)
            else:
                print("Price is less than average price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

execute order quantity: 1000 current executed: 71000
Successful execution for 0 th interval
execute order quantity: 1000 current executed: 72000
Successful execution for 1 th interval
execute order quantity: 1000 current executed: 73000
Successful execution for 2 th interval
execute order quantity: 1000 current executed: 74000
Successful execution for 3 th interval
execute order quantity: 1000 current executed: 75000
Successful execution for 4 th interval
execute order quantity: 1000 current executed: 76000
Successful execution for 5 th interval
execute order quantity: 1000 current executed: 77000
Successful execution for 6 th interval
execute order quantity: 1000 current executed: 78000
Successful execution for 7 th interval
execute order quantity: 1000 current executed: 79000
Successful execution for 8 th interval
execute order quantity: 1000 current executed: 80000
Successful execution for 9 th interval
execute order quantity: 1000 current executed: 81000
Successful execution for 10

KeyboardInterrupt: 

In [44]:
# convert the list of prices into a DataFrame
df4 = pd.DataFrame(batch4_prices)
df4

,Timestamp,Order_ID,Instrument,Price
0,2023-05-17 22:17:45.617137,243.0,EUR_CHF,0.97443
1,2023-05-17 22:17:50.731520,245.0,EUR_CHF,0.97442
2,2023-05-17 22:17:55.833569,247.0,EUR_CHF,0.97442
3,2023-05-17 22:18:00.948134,249.0,EUR_CHF,0.97442
4,2023-05-17 22:18:06.063477,251.0,EUR_CHF,0.97442
5,2023-05-17 22:18:11.250607,253.0,EUR_CHF,0.97442
6,2023-05-17 22:18:16.382364,255.0,EUR_CHF,0.97441
7,2023-05-17 22:18:21.504515,257.0,EUR_CHF,0.97441
8,2023-05-17 22:18:26.625826,259.0,EUR_CHF,0.97443
9,2023-05-17 22:18:31.744474,261.0,EUR_CHF,0.97443


In [45]:
# concatenate the two dataframes vertically
final_df = pd.concat([updated_df, df4], axis=0)
# calculate the total average execution price
average_execution_price4 = updated_df['Price'].sum() / len(updated_df)

average_execution_price4
current_executed_quantity
non_executed_quantity_final = total_order_quantity - current_executed_quantity

non_executed_quantity_final

0

In [46]:
df4.to_csv("Long_Interval_4.csv", index=False)
final_df.to_csv("Long_Final_Executed_Data.csv", index=False)

In [47]:
# Create a dictionary with your data
data = {'Interval': ['Interval 1', 'Interval 2', 'Interval 3', 'Interval 4'],
        'Executed Quantity': [200000, 10000, 0, 0], 
        'Percentage of Executed Quantity': [100, 3.33, 0, 0],
        'Non Executed Quantity': [0, 290000, 200000, 300000],
        'Percentage of Non Executed Quantity': [0, 96.67, 100, 100]}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)

# Add some sample data for the last three columns
df['Average Execution Price'] = [average_execution_price1, average_execution_price2, average_execution_price3, average_execution_price4]

# select columns to sum
sum_cols = ['Executed Quantity', 'Non Executed Quantity']

# select last column to calculate average
avg_col = 'Average Execution Price'

# calculate the total and average values
total = df[sum_cols].sum()
average = df[avg_col].mean()

# create a new row with the calculated values
new_row = pd.DataFrame({"Interval": "Total Vectors",
                        'Executed Quantity': [total['Executed Quantity']], 
                        'Percentage of Executed Quantity': int(total['Executed Quantity'] / 1000000 * 100),
                        'Non Executed Quantity': [total['Non Executed Quantity']],
                        'Percentage of Non Executed Quantity': int(total['Non Executed Quantity'] / 1000000 * 100),
                        'Average Execution Price': [average] })

# concatenate the original DataFrame with the new row
df = pd.concat([df, new_row])

# reset the index
df = df.reset_index(drop=True)

# Convert the DataFrame to a CSV file
df.to_csv('Long - Execution Output Vectors.csv', index=False)

In [50]:
from oandapyV20.endpoints.pricing import PricingInfo

def get_current_price(instrument):
    params = {"instruments": instrument}
    pricing_info = PricingInfo(accountID, params=params)
    response = client.request(pricing_info)
    
    for price in response["prices"]:
        if price["instrument"] == instrument:
            return float(price["bids"][0]["price"])

In [51]:
# 30 minutes after the end of the last execution window
print("Waite for 30 mins...")
time.sleep(1800)
#get current prices
current_long_price = get_current_price("EUR_CHF")

Waite for 30 mins...


In [53]:
# For LONG 
if current_long_price >= average_execution_price4:
    long_units = 0.5 * non_executed_quantity_final
    long_order = orders.OrderCreate(accountID, data={
        "order": {
            "instrument": "EUR_CHF",
            "units": long_units,
            "type": "MARKET",
            "side": "buy"
        }
    })
    response = client.request(long_order)
    print("LONG order placed:", response)
else:
    print("None")

None


In [54]:
import oandapyV20
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
client = oandapyV20.API(access_token=access_token)

In [55]:
#60min_Long
executed_50_percent = True
position = 'long'

def place_order(instrument, units):
    mo = MarketOrderRequest(instrument=instrument, units=units)
    r = orders.OrderCreate(accountID, data=mo.data)
    client.request(r)

def place_twap_order(instrument, total_units, ratio):
    units_to_execute = int(total_units * ratio)
    order_quantity = 20000  
    orders_to_place = units_to_execute // order_quantity
    for _ in range(orders_to_place):
        place_order(instrument, order_quantity)
        time.sleep(60)
        
def check_condition_and_execute_order():
    if position == 'long':
        price = get_current_price(instrument)
        print(f"Current Price: {price}")  # Print the current price  
#Different situations:
        if executed_50_percent and (price >= average_execution_price4):
            print("Placing LONG order - Situation 1")  
            place_order(instrument, non_executed_quantity_final * 0.5)
        elif executed_50_percent and (price < average_execution_price4):
            print("Placing LONG order - Situation 2") 
            place_order(instrument, non_executed_quantity_final * 0.5)
        elif not executed_50_percent and (price < average_execution_price4):
            print("Placing LONG TWAP order - Situation 3") 
            place_twap_order(instrument, current_executed_quantity, 0.2)
        else:
            print("Completed")  # Till no situations are met

In [ ]:
time.sleep(3600)
iteration_time = 60  # iteration times
for iteration in range(iteration_time):
    print(f"Checking conditions (iteration {iteration + 1} of {iteration_time})")
    check_condition_and_execute_order()
    time.sleep(60)  # Sleep for 1 minute before checking again